In [56]:
## Need to exclude states that ren't in the continental US

import pandas as pd
import numpy as np

df = pd.read_csv("../input/DataRecords.csv")
fips = pd.read_csv("../input/state_fips.csv")

df_joined = df.merge(fips, left_on="statefp", right_on="STATE", how='left')
display(df_joined.head())

df_joined = df_joined[~df_joined["STATE_NAME"].isin(["Alaska"])]
df_joined = df_joined[df_joined['EP_ASTHMA'].notna()]

df_joined.groupby("STATE")["EP_ASTHMA"].mean()
df_joined["STATE"].nunique()

#df_joined[df_joined["STATE"] == 2]

,OBJECTID,statefp,countyfp,tractce,affgeoid,geoid,name,COUNTY,StateAbbr,StateDesc,...,EPL_ASTHMA,EPL_CANCER,EPL_DIABETES,EPL_MHLTH,SHAPE__Length,SHAPE__Area,STATE,STUSAB,STATE_NAME,STATENS
0,1,1,1,20100,1400000US01001020100,1001020100,201.0,Autauga,AL,Alabama,...,0.54,0.49,0.52,0.63,"15,873.70","9,883,863.50",1,AL,Alabama,1779775
1,2,1,1,20200,1400000US01001020200,1001020200,202.0,Autauga,AL,Alabama,...,0.86,0.27,0.83,0.82,"9,764.61","3,325,691.60",1,AL,Alabama,1779775
2,3,1,1,20300,1400000US01001020300,1001020300,203.0,Autauga,AL,Alabama,...,0.72,0.52,0.66,0.77,"10,489.20","5,379,045.63",1,AL,Alabama,1779775
3,4,1,1,20400,1400000US01001020400,1001020400,204.0,Autauga,AL,Alabama,...,0.37,0.88,0.56,0.41,"12,162.23","6,355,819.12",1,AL,Alabama,1779775
4,5,1,1,20500,1400000US01001020500,1001020500,205.0,Autauga,AL,Alabama,...,0.43,0.43,0.33,0.50,"16,890.07","11,462,115.49",1,AL,Alabama,1779775


8

In [30]:
import requests

res = requests.get("https://redivis.com/api/v1/datasets/EIDC.cdc_places", headers={"AUTHORIZATION": "asdfghjkl"})
print(res.json())

{'error': {'status': 401, 'message': 'Invalid credentials provided'}}
